# Features Ale

La idea de esta notebook es crear los siguientes features:

 - [Rango_horario](#Rango_horario)
 - [Prob_compra_x_usuario_x_dia](#Prob_compra_x_usuario_x_dia)
 - [Prob_compra_x_dia](#Prob_compra_x_dia)

In [1]:
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

limpieza_path = os.path.join(current_folder, '..', 'Limpieza.ipynb')

In [3]:
%run $limpieza_path
df = get_clean_df()

Creo el nuevo dataframe que va a tener una fila por usuario:

In [4]:
df_features = pd.DataFrame(index=df['person'].unique())
df_features.index.name = 'person'

In [5]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


#### Prob_compra_x_usuario_x_dia

In [6]:
df_eventos_aux = df.loc[df['event']=='conversion'].copy()

In [7]:
df_eventos_aux = df_eventos_aux[['event','timestamp','person']]

In [8]:
df_eventos_aux['dia'] = pd.to_datetime(df_eventos_aux['timestamp']).dt.weekday_name

In [9]:
df_eventos_aux['hora'] = pd.to_datetime(df_eventos_aux['timestamp']).dt.hour

In [10]:
df_compras_x_dia = df_eventos_aux.groupby(['person','dia']).size().reset_index(name='cant_compras_x_dia')

In [11]:
df_compras_x_dia.head()

,person,dia,cant_compras_x_dia
0,000ba417,Saturday,1
1,001001be,Wednesday,1
2,001804a2,Friday,1
3,0019e639,Friday,1
4,0019e639,Tuesday,1


In [12]:
df_dias_c = df_eventos_aux.groupby(['person','event']).size().reset_index(name='cant_compras_totales')

In [13]:
df_dias_c.head()

,person,event,cant_compras_totales
0,000ba417,conversion,1
1,001001be,conversion,1
2,001804a2,conversion,1
3,0019e639,conversion,2
4,001b0bf9,conversion,1


In [14]:
Prob_compra_x_usuario_x_dia = pd.merge(df_compras_x_dia,df_dias_c,how="inner",on='person')

In [15]:
Prob_compra_x_usuario_x_dia.head()

,person,dia,cant_compras_x_dia,event,cant_compras_totales
0,000ba417,Saturday,1,conversion,1
1,001001be,Wednesday,1,conversion,1
2,001804a2,Friday,1,conversion,1
3,0019e639,Friday,1,conversion,2
4,0019e639,Tuesday,1,conversion,2


In [16]:
Prob_compra_x_usuario_x_dia['prob_compra_por_dia'] = Prob_compra_x_usuario_x_dia['cant_compras_x_dia'] / Prob_compra_x_usuario_x_dia['cant_compras_totales']

In [17]:
del Prob_compra_x_usuario_x_dia['cant_compras_x_dia']
del Prob_compra_x_usuario_x_dia['cant_compras_totales']


In [18]:
Prob_compra_x_usuario_x_dia.head()

,person,dia,event,prob_compra_por_dia
0,000ba417,Saturday,conversion,1.0
1,001001be,Wednesday,conversion,1.0
2,001804a2,Friday,conversion,1.0
3,0019e639,Friday,conversion,0.5
4,0019e639,Tuesday,conversion,0.5


#### Rango_horario

In [19]:
Rango_horario = df_eventos_aux.copy()

In [20]:
Rango_horario['madrugada'] = np.where(((Rango_horario['hora']<7) & (Rango_horario['hora']>=0)), 1,0)
Rango_horario['maniana'] = np.where(((Rango_horario['hora']<13) & (Rango_horario['hora']>=7)), 1,0)
Rango_horario['tarde'] = np.where(((Rango_horario['hora']<19) & (Rango_horario['hora']>=13)), 1,0)
Rango_horario['noche'] = np.where(((Rango_horario['hora']<24) & (Rango_horario['hora']>=19)), 1,0)

In [21]:
Rango_horario.head()

,event,timestamp,person,dia,hora,madrugada,maniana,tarde,noche
195,conversion,2018-05-18 01:12:20,49c19e32,Friday,1,1,0,0,0
234,conversion,2018-05-18 01:05:23,49c19e32,Friday,1,1,0,0,0
724,conversion,2018-05-29 07:51:14,39df97e0,Tuesday,7,0,1,0,0
744,conversion,2018-05-29 07:13:30,380c0e60,Tuesday,7,0,1,0,0
1198,conversion,2018-05-29 19:31:46,35ee0cc8,Tuesday,19,0,0,0,1


#### Prob_compra_x_dia

In [22]:
df_rel_person_dia = df_eventos_aux[['person','dia']]

In [23]:
df_rel_person_dia.head()

,person,dia
195,49c19e32,Friday
234,49c19e32,Friday
724,39df97e0,Tuesday
744,380c0e60,Tuesday
1198,35ee0cc8,Tuesday


In [24]:
compras_x_dia = df_eventos_aux.groupby('dia').size().reset_index(name='cant_compras_x_dia')

In [25]:
compras_x_dia

,dia,cant_compras_x_dia
0,Friday,894
1,Monday,865
2,Saturday,591
3,Sunday,470
4,Thursday,1425
5,Tuesday,1395
6,Wednesday,1451


In [26]:
total_ventas = (df_eventos_aux.shape)[0]

In [27]:
Prob_compra_x_dia = pd.merge(df_rel_person_dia,compras_x_dia,how="inner",on='dia')

In [28]:
Prob_compra_x_dia.head()

,person,dia,cant_compras_x_dia
0,49c19e32,Friday,894
1,49c19e32,Friday,894
2,eca84f0a,Friday,894
3,eb40c8c1,Friday,894
4,eca84f0a,Friday,894


In [29]:
Prob_compra_x_dia['prob_compra_por_dia'] = Prob_compra_x_dia['cant_compras_x_dia'] / total_ventas

In [30]:
del Prob_compra_x_dia['cant_compras_x_dia']

In [31]:
Prob_compra_x_dia.head()

,person,dia,prob_compra_por_dia
0,49c19e32,Friday,0.126075
1,49c19e32,Friday,0.126075
2,eca84f0a,Friday,0.126075
3,eb40c8c1,Friday,0.126075
4,eca84f0a,Friday,0.126075


Se agregan al df_features

In [32]:
df_features = pd.merge(df_features, Prob_compra_x_dia,how='left', on='person')

In [33]:
df_features = pd.merge(df_features, Rango_horario,how='left', on='person')

In [34]:
df_features = pd.merge(df_features, Prob_compra_x_usuario_x_dia,how='left', on='person')